In [6]:
import shutil
import argparse
import os
import torch
from torch import distributed, nn
import uvicorn
from fastapi import FastAPI, HTTPException, Request, Response, Form
from contextlib import asynccontextmanager
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
from typing import Dict, List, Literal, Optional, Union
import time 
import numpy as np 
import logging
import uuid
from modelscope import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [9]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, GenerationConfig


MODEL_PATH = "/root/test/model_zoo/Qwen1.5-7B-Chat"
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, device_map="auto", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
generation_config = GenerationConfig.from_pretrained(MODEL_PATH)


# async def translate_prompt(prompt):
#     prefix = '请把下面一段话翻译成英文；如果原本就是英文那么请直接输出原文：\n'
#     full_prompt = prefix + prompt + "\n翻译成英文结果："
#     print(full_prompt)
#     inputs = tokenizer(full_prompt,return_tensors='pt').to('cuda')
#     generate_ids = model.generate(inputs.input_ids, generation_config=generation_config)
#     pred = tokenizer.batch_decode(generate_ids,skip_special_tokens=True)
#     print("\npred:\n")
#     print(pred)
#     return pred[0].split("翻译成英文结果：")[-1]

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.15it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def translate_prompt(prompt):
    prefix = '请把下面一段话翻译成英文；如果原本就是英文那么请直接输出原文：\n'
    full_prompt = prefix + prompt + "\n翻译成英文结果："
    print(full_prompt)
    inputs = tokenizer(full_prompt,return_tensors='pt').to('cuda')
    generate_ids = model.generate(inputs.input_ids, generation_config=generation_config)
    pred = tokenizer.batch_decode(generate_ids,skip_special_tokens=True)
    print("\npred:\n")
    print(pred)
    return pred[0].split("翻译成英文结果：")[-1]

In [14]:
translate_prompt("кот играет в снегу")

请把下面一段话翻译成英文；如果原本就是英文那么请直接输出原文：
кот играет в снегу
翻译成英文结果：

pred:

['请把下面一段话翻译成英文；如果原本就是英文那么请直接输出原文：\nкот играет в снегу\n翻译成英文结果：']


''

In [15]:
import torch
from transformers import MarianMTModel, AutoTokenizer, pipeline


class Translator:

    def __init__(self, model_path):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = MarianMTModel.from_pretrained(model_path)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = self.model.to(self.device)
        # self.pipeline = pipeline("translation", model=self.model, tokenizer=self.tokenizer)

    def translate(self, text):
        model_inputs = self.tokenizer(text, return_tensors="pt", padding=True).to(self.device)
        translated_outputs = self.model.generate(**model_inputs, num_beams=10)
        return [self.tokenizer.decode(_, skip_special_tokens=True) for _ in translated_outputs]

In [18]:
model = Translator("/root/test/nlp/mt_zh2en/mt_zh2en/multi_en_models/opus-mt-ja-en")
# res = model.translate(["有时候，有时候，我会", "你是一个好人"])
# res = model.translate(["sometimes, sometimes, I would", "you are a good guy"])

In [17]:
!pip3 install sacremoses

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


^C
ERROR: Operation cancelled by user


In [19]:
res = model.translate(["こんにちはようこそ", "雪の中で遊ぶ猫"])

In [20]:
res

['Hello. Welcome.', "It's a cat playing in the snow."]

In [ ]:
python main2.py --trmodelpath /root/test/nlp/mt_zh2en/mt_zh2en/multi_en_models/opus-mt-fr-en